In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('transformations_actions')
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/02 06:33:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# 스파크 설정 환경 확인
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.app.id', 'local-1685687626684'),
 ('spark.driver.host', 'ip-172-31-44-191.ap-northeast-3.compute.internal'),
 ('spark.driver.port', '44927'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'transformations_actions'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.startTime', '1685687624930')]

Python 리스트를 RDD로 만들기

In [3]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

1. RDD 내의 모든 데이터 가져오기
- collect()

In [4]:
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

2. 각 데이터 별 개수 세기
- countByValue()

In [5]:
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

3. 상위 n 개의 데이터 가져오기
- take(n)

In [6]:
foods.take(3)

['짜장면', '마라탕', '짬뽕']

4. 처음 1개의 데이터만 가져오기
- first()


In [7]:
foods.first()

'짜장면'

5. RDD내의 데이터 개수 세기

In [8]:
foods.count()

12

6. 중복데이터 제거 (Transformarions)
- distinct

In [9]:
fd = foods.distinct()
fd

PythonRDD[9] at RDD at PythonRDD.scala:53

In [10]:
fd.collect() #Action

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [11]:
# 중복을 제외한 개수는 distinct 후 count
foods.distinct().count()

7

* 중요한 기능은 아님
- 요소를 하나씩 꺼내서 함수에 저장할 때 사용
- foreach
    worker에게만 일을 시켜 master에게 리턴이 되지 않음
    연산의 결과 등을 로그화 시킬 때 사용

In [16]:
foods.foreach(lambda x : print(x+'냠냠')) #master에서 강제로 띄워주려 print

짜장면냠냠
마라탕냠냠
짬뽕냠냠
떡볶이냠냠
쌀국수냠냠
짬뽕냠냠
짜장면냠냠
짜장면냠냠
짜장면냠냠
라면냠냠
우동냠냠
라면냠냠


In [15]:
foods.foreach(lambda x : x) # 워커에서만 태스크가 일어남

Narrow Transformation
- 1:1 변환을 의미
- 하나의 열을 조작하기 위해 다른 열 및 파티션의 데이터를 사용하지 않는다.

filter() map() flatMap() sample() union()

In [18]:
sample_rdd = sc.parallelize([1,2,3])
sample_rdd

ParallelCollectionRDD[20] at readRDDFromFile at PythonRDD.scala:274

In [19]:
sample_rdd2 = sample_rdd.map(lambda x : x+2)

In [20]:
sample_rdd2.collect()

[3, 4, 5]

In [ ]:
flatMap(<func>)
- map

In [21]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [22]:
moviesRDD = sc.parallelize(movies)
moviesRDD

ParallelCollectionRDD[22] at readRDDFromFile at PythonRDD.scala:274

In [23]:
mapMovies = moviesRDD.map(lambda x : x.split())

In [24]:
mapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [25]:
flatMovies = moviesRDD.flatMap(lambda x : x.split())
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

Wide Transformation

In [26]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[25] at readRDDFromFile at PythonRDD.scala:274

- groupBy()



`groupBy(<func>)`
- `<func>`: 기준을 마련하는 함수

In [28]:
foodsGroup = foods.groupBy(lambda x : x[0]) # 기준을 마련해줌
foodsGroup.collect() #key Value RDD 가 나옴 

#ResultIterable 여러개의 결과를 가지고 있음

[('짜', <pyspark.resultiterable.ResultIterable at 0x7f2688d4ba60>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7f2688cf44c0>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7f2688ba9040>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7f2688ba9250>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7f2688ba90d0>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7f2688ba91c0>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7f2688ba9670>)]

In [29]:
res = foodsGroup.collect()
res

[('짜', <pyspark.resultiterable.ResultIterable at 0x7f2688a9dd90>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7f2688a9dfd0>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7f2688d51eb0>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7f2688cab070>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7f2688de0af0>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7f26a0041ee0>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7f26a0041730>)]

In [30]:
for (k,v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']


In [31]:
sc.stop()

In [ ]:
# spark context를 다시 실행하려면 stop을 하고 새로 만들어야함 
